# 黑色金属 - 螺纹钢

In [1]:
# 设置工作目录
import os
os.getcwd()
if os.name == 'posix':
    os.chdir('/Volumes/Repository/Projects/ffa/')
else:
    os.chdir("E:\\Document\\Project\\ffa")

In [2]:
# 加载依赖模块
import pandas as pd
import numpy as np
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import importlib
import commodity
import json
from datetime import timedelta
    

In [18]:
# 重新加载salary计算模块
importlib.reload(commodity)

<module 'commodity' from '/Volumes/Repository/Projects/ffa/commodity.py'>

## 数据准备与预处理

In [3]:
symbol_id = 'RB'
symbol_name = '螺纹钢'
fBasePath = 'steel/data/mid-stream/螺纹钢/'
json_file = './steel/setting.json'

### 数据索引设置

In [18]:
# 品种数据索引初始化
# 首次使用json配置文件存取品种的数据索引
data_index = {'主力合约': {'Name': "dominant_contract", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dominant_contract', 'DataFrame': "df_basis"},
                '近月合约': {'Name': "near_contract", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'near_contract', 'DataFrame': "df_basis"},
                '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货收盘价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货结算价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                '现货价格': {'Name': "spot_price", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'spot_price', 'DataFrame': "df_basis"},
                '基差': {'Name': "basis", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dom_basis', 'DataFrame': "df_basis"},
                '基差率': {'Name': "basis_rate", 'Source':'AKShare', 'Path': fBasePath + '螺纹钢基差.xlsx', 'Field':'dom_basis_rate', 'DataFrame': "df_basis"},
                '产量': {'Name': "production", 'Source':'Choice', 'Path': fBasePath + '螺纹钢产量.xlsx', 'Field':'产量：钢筋：全国：当月值', 'DataFrame': "df_prodoction"},
                '销量': {'Name': "sales", 'Source':'Choice', 'Path': fBasePath + '螺纹钢销量.xlsx', 'Field':'销量：钢筋：累计值', 'DataFrame': "df_sales"},
                '库存': {'Name': "instock", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'库存：螺纹钢：合计', 'DataFrame': "df_instock"},
                # TODO: 校验Choice提供的另外一套库存和仓单数据
                '仓单': {'Name': "receipt", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'仓单数量：螺纹钢', 'DataFrame': "df_instock"}
                }
profit_formula = {'Name': 'profit_formula', 'Factor': {'铁矿石': 1.6, '焦炭': 0.6}, '其他成本': 1200}
dominant_months = [1, 5, 10]
symbol_setting = {'DataIndex': data_index,
                  'ProfitFormula': profit_formula,
                  'DominantMonths': dominant_months}


In [4]:
# 构造品种数据访问对象
# symbol = commodity.SymbolData(symbol_id, symbol_name, json_file, symbol_setting)
symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
symbol.merge_data()
symbol.get_spot_months()
symbol_j = commodity.SymbolData('J', '焦炭', json_file)
symbol_j.merge_data()
symbol_i = commodity.SymbolData('I', '铁矿石', json_file)
symbol_i.merge_data()

,日期,主力合约,近月合约,现货价格,基差,基差率,主力合约收盘价,主力合约结算价,产量,库存,仓单,库存消费比
2527,2011-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9850,NaN,NaN
2469,2011-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12755.080582,9850,NaN,NaN
2528,2011-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9860,NaN,NaN
2529,2011-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9800,NaN,NaN
2530,2011-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9770,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2461,2023-11-27,i2401,i2312,1008.33,31.83,0.031567,980.5,976.5,NaN,11310,2200,NaN
2462,2023-11-28,i2401,i2312,1003.22,43.22,0.043081,951,960,NaN,11310,2200,NaN
2463,2023-11-29,i2401,i2312,997.00,40.50,0.040622,955.5,956.5,NaN,11310,2200,NaN
2464,2023-11-30,i2401,i2312,988.89,32.39,0.032754,969.5,956.5,NaN,11499,2200,NaN


In [5]:
symbol_chain = commodity.SymbolChain('Steel', '黑色金属', json_file)
symbol_chain.add_symbol(symbol)
symbol_chain.add_symbol(symbol_i)
symbol_chain.add_symbol(symbol_j)
df_profit_c = symbol.get_profits(symbol_chain)
# df_profit_c.dropna(axis=0, how='all', subset=['现货利润', '盘面利润'], inplace=True)
symbol.calculate_data_rank(trace_back_months=60)

,日期,库存历史时间百分位,库存历史时间分位,仓单历史时间百分位,仓单历史时间分位,现货利润历史时间百分位,现货利润历史时间分位,盘面利润历史时间百分位,盘面利润历史时间分位
0,2018-12-03,0.004808,1,0.158155,1,0.923651,5,0.500412,3
1,2018-12-04,0.004808,1,0.158155,1,0.914523,5,0.403133,3
2,2018-12-05,0.004808,1,0.158155,1,0.904564,5,0.732894,4
3,2018-12-06,0.000801,1,0.158155,1,0.900415,5,0.754328,4
4,2018-12-07,0.000801,1,0.158155,1,0.880498,5,0.711459,4
...,...,...,...,...,...,...,...,...,...
1243,2023-11-27,0.106571,1,0.285832,2,0.156846,1,0.001649,1
1244,2023-11-28,0.106571,1,0.285832,2,0.107884,1,0.000824,1
1245,2023-11-29,0.106571,1,0.285832,2,0.107054,1,0.002473,1
1246,2023-11-30,0.110577,1,0.285832,2,0.095436,1,0.014015,1


In [68]:
if 'ProfitFormula' not in symbol.symbol_setting:
    pass

profit_formula = symbol.symbol_setting['ProfitFormula']
cost_factors = profit_formula['Factor']

#df_spot_price = symbol.symbol_data[['日期', '现货价格']]
df_price = symbol.symbol_data[['日期', '现货价格', '主力合约结算价']]

# df_raw_materials = pd.DataFrame()
df_raw_materials = pd.DataFrame(columns=['日期', '原材料现货成本总和', '原材料盘面成本总和'])
first_combine = True
for raw_material, multiplier in cost_factors.items():
    df_raw_material = symbol_chain.get_symbol(raw_material).symbol_data[['日期', '现货价格', '主力合约结算价']].copy()
    df_raw_material.dropna(axis=0, how='all', subset=['现货价格', '主力合约结算价'], inplace=True)
    df_raw_material['现货成本'] = df_raw_material['现货价格'] * multiplier
    df_raw_material['盘面成本'] = df_raw_material['主力合约结算价'] * multiplier
    df_raw_materials = pd.merge(df_raw_materials, df_raw_material, 
                                on='日期', how='outer')
    if first_combine:
        df_raw_materials['原材料现货成本总和'] = df_raw_materials['现货成本']
        df_raw_materials['原材料盘面成本总和'] = df_raw_materials['盘面成本']     
        first_combine = False   
    #     df_raw_materials['原材料现货成本总和'] = 0
    #     df_raw_materials['原材料盘面成本总和'] = 0
    else:
    #     df_raw_materials['原材料现货成本总和'].fillna(0, inplace=True)
    #     df_raw_materials['原材料盘面成本总和'].fillna(0, inplace=True)
        df_raw_materials['原材料现货成本总和'] = df_raw_materials['原材料现货成本总和'] + df_raw_materials['现货成本']
        df_raw_materials['原材料盘面成本总和'] = df_raw_materials['原材料盘面成本总和'] + df_raw_materials['盘面成本']
    df_raw_materials= df_raw_materials[['日期', '原材料现货成本总和', '原材料盘面成本总和']]

df_profit = pd.merge(df_raw_materials, df_price, on='日期', how='outer')
df_profit['现货利润'] = df_profit['现货价格'] - df_profit['原材料现货成本总和']- profit_formula['其他成本']
df_profit['盘面利润'] = df_profit['主力合约结算价'] - df_profit['原材料盘面成本总和']- profit_formula['其他成本']
symbol.symbol_data = pd.merge(symbol.symbol_data, df_profit, on='日期', how='outer')
# df_all = symbol.symbol_data

In [70]:
merged_data = symbol.merge_data()
# 生成上由原材料的现货和期货价格数据
symbol_j = commodity.SymbolData('J', '焦炭', json_file)
symbol_j.merge_data()
symbol_i = commodity.SymbolData('I', '铁矿石', json_file)
symbol_i.merge_data()
profit_j = symbol_j.symbol_data[['日期', '现货价格', '主力合约结算价']].copy()
profit_j.rename(columns={'现货价格': symbol_j.name+'现货', '主力合约结算价': symbol_j.name+'期货'}, inplace=True)
profit_i = symbol_i.symbol_data[['日期', '现货价格', '主力合约结算价']].copy()
profit_i.rename(columns={'现货价格': symbol_i.name+'现货', '主力合约结算价': symbol_i.name+'期货'}, inplace=True)
# 计算品种的现货利润和盘面利润
formula = symbol.symbol_setting['ProfitFormula']
df_profit_b = pd.merge(profit_j, profit_i, on='日期', how='outer')
# df_profit_b = pd.merge(symbol.symbol_data[['日期', '现货价格', '主力合约结算价']], df_profit_b, on='日期', how='outer').dropna(axis=0, how='all', subset=['现货价格', '主力合约结算价'])
df_profit_b = pd.merge(symbol.symbol_data[['日期', '现货价格', '主力合约结算价']], df_profit_b, on='日期', how='outer')
df_profit_b['现货利润'] = df_profit_b['现货价格'] - formula['Factor'][symbol_j.name] * df_profit_b[symbol_j.name+'现货'] - formula['Factor'][symbol_i.name] * df_profit_b[symbol_i.name+'现货'] - formula['其他成本']
df_profit_b['盘面利润'] = df_profit_b['主力合约结算价'] - formula['Factor'][symbol_j.name] * df_profit_b[symbol_j.name+'期货'] - formula['Factor'][symbol_i.name] * df_profit_b[symbol_i.name+'期货'] - formula['其他成本']
# df_profit.dropna(axis=0, how='all', subset=['现货利润', '盘面利润'], inplace=True)
# df_append = df_profit_b[['日期', '现货利润', '盘面利润']].copy()

In [69]:
df_profit_b.to_excel('data/profit_rb.xlsx')

In [ ]:
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
trade_date


### 更新数据

- 通过AKshare接口更新现货价格数据
- 手动更新Choice导出数据（使用Microsoft Excel打开，自动更新后保存）

In [7]:
# 更新现货价格数据 - AKShare
# 追加最新数据
df_basis = symbol.update_akshare_file(mode='append')
# 2011、2012年数据待更新
# symbol_data.update_akshare_file(mode='period', start_date='20120101', end_date='20121231')

/Volumes/Repository/app/miniconda3/lib/python3.10/site-packages/akshare/futures/futures_basis.py:101: UserWarning: 20231202非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")


## 基差-库存/仓单-利润分析

### 历史走势分析

前置条件：
- 非季节性品种确认

分析内容：
- 现货价格/期货价格（收盘价）、基差的历史趋势
- 基差率历史趋势，基差率历史分位
- 库存、仓单、库存消费比，库存、仓单的历史分位
- 现货利润和盘面利润，及其历史分位
- 现货月区域标记
- 多维指标共振区域标记

扩展功能：
- 图表可配置化
- 小图：时点跨期套利分析
- 小图：时点期限结构分析

In [ ]:
# 图表配置信息
figure_dict = {'基础属性':{'title':'基差-库存-利润分析', 'width':'auto', 'height':800, 'slider':False, 'selector':True},
               '主图':{'title':'基差', 'height':0.7, 'x_axis':symbol.symbol_data['日期'], 
                      'y_axis':{'期货收盘价': {'type':'Scatter', 'color':'rgb(84,134,240)', 'secondary_y':False},
                                '现货价格':{'type':'Scatter', 'color':'rgb(84,134,240)', 'secondary_y':False},
                                '基差':{'type':'Area', 'color':'rgb(239,181,59)', 'secondary_y':True, 'stackgroup':'one'}},
               '辅图':{'库存':{'title':'基差', 'height':0.7, 'x_axis':symbol.symbol_data['日期'],
                      'y_axis':{'库存':{'type':'Bar', 'color':'rgb(84,134,240)', 'secondary_y':False}}},
                      '利润':{}}
               }}

In [ ]:
symbol.get_spot_months()
symbol.spot_months

In [45]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                   vertical_spacing=0.01, 
                   subplot_titles=('基差分析', '基差率', '库存/仓单历史分位', '现货利润/盘面利润'), 
                   row_width=[0.1, 0.1, 0.1, 0.7])

# 创建主图：期货价格、现货价格、基差
fig_future_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['主力合约收盘价'], name='期货价格', 
                              marker_color='rgb(84,134,240)')
fig_spot_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['现货价格'], name='现货价格', marker_color='rgb(105,206,159)')
fig_basis = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['基差'], stackgroup='one', name='基差', 
                       marker_color='rgb(239,181,59)', showlegend=False)
fig.add_trace(fig_basis, secondary_y=True)
fig.add_trace(fig_future_price, row = 1, col = 1)
fig.add_trace(fig_spot_price, row = 1, col = 1)

# 创建辅图-基差率，并根据基差率正负配色
sign_color_mapping = {0:'green', 1:'red'}
fig_basis_rate = go.Bar(x=symbol.symbol_data['日期'], y = symbol.symbol_data['基差率'], name='基差率',
                        marker=dict(color=symbol.basis_color['基差率颜色'], colorscale=list(sign_color_mapping.values()),
                                    showscale=False),
                        showlegend=False,
                        hovertemplate='%{y:.2%}')
fig.add_trace(fig_basis_rate, row = 2, col = 1)

# 创建辅图-库存/仓单
fig_receipt = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['仓单'], name='仓单', marker_color='rgb(239,181,59)')
fig_storage = go.Bar(x=symbol.symbol_data['日期'], y=symbol.symbol_data['库存'], name='库存', marker_color='rgb(234,69,70)')
fig.add_trace(fig_receipt, row = 3, col = 1, secondary_y=True)
fig.add_trace(fig_storage, row = 3, col = 1)

# 创建辅图-库存/仓单历史时间百分位，并根据分位配色
histroy_color_mapping ={1:'red', 2:'lightblue', 3:'lightblue', 4:'lightblue', 5:'green'}
# df_rank['仓单分位颜色'] = df_rank['仓单历史时间分位'].map(histroy_color_mapping)
# fig_receipt_rank = go.Scatter(x=df_rank['日期'], y=df_rank['仓单历史时间百分位'], name='仓单分位', marker_color='rgb(239,181,59)')
fig_receipt_rank = go.Scatter(x=symbol.data_rank['日期'], y=symbol.data_rank['仓单历史时间百分位'], name='仓单分位', mode='markers',
                              marker=dict(size=2, color=symbol.data_rank['仓单历史时间分位'], colorscale=list(histroy_color_mapping.values())),
                              showlegend=False,
                              hovertemplate='%{y:.2%}')
# fig.add_trace(fig_receipt_rank, row = 3, col = 1, secondary_y=True)
symbol.data_rank['库存分位颜色'] = symbol.data_rank['库存历史时间分位'].map(histroy_color_mapping)
# fig_storage_rank = go.Bar(x=df_rank['日期'], y=df_rank['库存历史时间百分位'], name='库存分位', marker_color='rgb(234,69,70)')
fig_storage_rank = go.Bar(x=symbol.data_rank['日期'], y=symbol.data_rank['库存历史时间百分位'], name='库存分位', marker_color=symbol.data_rank['库存分位颜色'],
                          hovertemplate='%{y:.2%}')
# fig.add_trace(fig_storage_rank, row = 3, col = 1)

# 创建辅图-现货利润/盘面利润
# fig_spot_profit = go.Scatter(x=df_profit['日期'], y=df_profit['现货利润'], name='现货利润', marker_color='rgb(239,181,59)')
# fig_future_profit = go.Bar(x=df_profit['日期'], y=df_profit['盘面利润'], name='盘面利润', marker_color='rgb(234,69,70)')
# fig.add_trace(fig_spot_profit, row = 4, col = 1, secondary_y=True)
# fig.add_trace(fig_future_profit, row = 4, col = 1)

# 创建辅图-现货利润/盘面利润历史时间分位
symbol.data_rank['盘面利润分位颜色'] = symbol.data_rank['盘面利润历史时间分位'].map(histroy_color_mapping)
fig_spot_profit = go.Scatter(x=symbol.data_rank['日期'], y=symbol.data_rank['现货利润历史时间百分位'], name='现货利润', mode='markers',
                             marker=dict(size=2, color=symbol.data_rank['现货利润历史时间分位'], colorscale=list(histroy_color_mapping.values())),
                             hovertemplate='%{y:.2%}')
fig.add_trace(fig_spot_profit, row = 4, col = 1, secondary_y=True)
fig_future_profit = go.Bar(x=symbol.data_rank['日期'], y=symbol.data_rank['盘面利润历史时间百分位'], name='盘面利润', marker_color=symbol.data_rank['盘面利润分位颜色'],
                           showlegend=False,
                           hovertemplate='%{y:.2%}')
fig.add_trace(fig_future_profit, row = 4, col = 1)



# 根据交易时间过滤空数据
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=symbol.symbol_data['日期'].iloc[0],end=symbol.symbol_data['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))

for _, row in symbol.spot_months.iterrows():
    fig.add_shape(
        # 矩形
        type="rect",
        # 矩形的坐标
        x0=row['Start Date'],
        x1=row['End Date'],
        y0=0,
        y1=1,
        xref='x',
        yref='paper',
        # 矩形的颜色和透明度
        fillcolor="LightBlue",
        opacity=0.1,
        # 矩形的边框
        line_width=0,
        # 矩形在数据之下
        layer="below"
    )

select_synchronize_index_value = ['基差率', '库存|仓单', '现货利润|盘面利润']
# df_signals =symbol.get_signals(select_synchronize_index_value)
symbol.signals = pd.merge(symbol.symbol_data[['日期', '基差率']],
                        symbol.data_rank[['日期', '库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']],
                        on='日期', how='outer')
symbol.signals['基差率'] = symbol.signals['基差率'].map(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
# print(symbol.signals)
# # For other columns
# for col in ['库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']:
#     symbol.signals[col] = symbol.signals[col].map(lambda x: -1 if x == 5 else (0 if x != 1 else 1))
for col in ['库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']:
    symbol.signals[col] = symbol.signals[col].map(lambda x: -1 if x == 5 else (0 if x != 1 else 1)).fillna(0).astype(int)
print(symbol.signals)
symbol.signals['库存|仓单'] = symbol.signals['库存历史时间分位'] | symbol.signals['仓单历史时间分位']
symbol.signals['现货利润|盘面利润'] = symbol.signals['现货利润历史时间分位'] | symbol.signals['盘面利润历史时间分位']
if len(select_synchronize_index_value)!=0:
    symbol.signals['信号数量'] = symbol.signals[select_synchronize_index_value].sum(axis=1)

signal_nums = len(select_synchronize_index_value)
df_short_signals = symbol.signals[symbol.signals['信号数量']==-signal_nums]        
for _, row in df_short_signals.iterrows():
    next_day = row['日期'] + timedelta(days=1)
    fig.add_shape(
        type='circle',
        x0=row['日期'], x1=next_day,
        y0=1, y1=0.99,
        xref='x', yref='paper',
        fillcolor='green',
        line_color='green'
    )

# X轴坐标按照年-月显示
fig.update_xaxes(
    showgrid=True,
    zeroline=True,
    dtick="M1",  # 按月显示
    ticklabelmode="period",   # instant  period
    tickformat="%b\n%Y",
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = False, # 下方滑动条缩放
    # 增加固定范围选择
    # rangeselector = dict(
    #     buttons = list([
    #         dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
    #         dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
    #         dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
    #         dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
    #         dict(step = 'all')
    #         ]))
)
#fig.update_traces(xbins_size="M1")
max_y = symbol.symbol_data['主力合约收盘价'] .max() * 1.05
min_y = symbol.symbol_data['主力合约收盘价'] .min() * 0.95
fig.update_layout(
    yaxis_range=[min_y,max_y],
    #autosize=False,
    #width=800,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor='WhiteSmoke',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis2_showgrid=False,
    hovermode='x unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

fig.show()

             日期  基差率  库存历史时间分位  仓单历史时间分位  现货利润历史时间分位  盘面利润历史时间分位
0    2010-03-29    0         0         0           0           0
1    2010-03-30    0         0         0           0           0
2    2010-03-31    0         0         0           0           0
3    2010-04-01    0         0         0           0           0
4    2010-04-02    0         0         0           0           0
...         ...  ...       ...       ...         ...         ...
3407 2023-11-27    1         1         0           1           1
3408 2023-11-28    1         1         0           1           1
3409 2023-11-29    1         1         0           1           1
3410 2023-11-30    1         1         0           1           1
3411 2023-12-01    1         1         0           0           0

[3412 rows x 6 columns]


In [33]:
self = symbol
df_rank = pd.DataFrame()
mode='time'
trace_back_months=3
quantiles=[0, 20, 40, 60, 80, 100]
ranks=[1, 2, 3, 4, 5]
data_list=['库存', '仓单', '现货利润', '盘面利润']

field = data_list[0]
df_append= pd.DataFrame()
df_append['日期'] = self.symbol_data['日期']
if trace_back_months == 'all':
    window_size = len(self.symbol_data)
else:
    window_size = trace_back_months * 20  # assuming 30 days per month
if mode=='time':
    value_field = field + '历史时间百分位'
    rank_field = field + '历史时间分位'
    df_append[value_field] = self.symbol_data[field].rolling(window=window_size, min_periods=1).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    quantiles = np.percentile(df_append[value_field].dropna(), quantiles)
elif mode=='value':
    value_field = field + '历史数值百分位'
    rank_field = field + '历史数值分位'
    df_append[value_field] = self.symbol_data[field].rolling(window=window_size, min_periods=1).apply(lambda x: (x[-1] - np.min(x)) / (np.max(x) - np.min(x)))
    quantiles = list(map(lambda x: x/100, quantiles))
else:
    None

df_append[rank_field] = pd.cut(df_append[value_field].dropna(), bins=quantiles, labels=ranks, include_lowest=True, duplicates='drop', right=False)
if df_rank.empty:
    df_rank = df_append
else:
    df_rank = pd.merge(df_rank, df_append, on='日期', how='outer')

df_rank = pd.merge(df_rank, symbol.symbol_data[['日期', field]], on='日期', how='outer')


In [31]:
self = symbol
df_rank = pd.DataFrame()
mode='time'
trace_back_months=1
quantiles=[0, 20, 40, 60, 80, 100]
ranks=[1, 2, 3, 4, 5]
data_list=['库存', '仓单', '现货利润', '盘面利润']
field = data_list[0]

df_append= pd.DataFrame()
df_append['日期'] = self.symbol_data['日期']
if mode=='time':
    value_field = field + '历史时间百分位'
    rank_field = field + '历史时间分位'
elif mode=='value':
    value_field = field + '历史数值百分位'
    rank_field = field + '历史数值分位'
else:
    None

if trace_back_months == 'all':
    window_size = len(self.symbol_data)
else:
    # Assuming that data is daily, convert months to days
    window_size = trace_back_months * 20

if mode=='time':
    df_append[value_field] = self.symbol_data[field].expanding().apply(lambda x: (x.rank(method='min') / len(x)).iloc[-1])
    quantiles = np.percentile(df_append[value_field].dropna(), quantiles)
elif mode=='value':
    df_append[value_field] = self.symbol_data[field].expanding().apply(lambda x: (x.iloc[-1] - x.min()) / (x.max() - x.min()))
    quantiles = list(map(lambda x: x/100, quantiles))

df_append[rank_field] = pd.cut(df_append[value_field].dropna(), bins=quantiles, labels=ranks, include_lowest=True, duplicates='drop', right=False)
if df_rank.empty:
    df_rank = df_append
else:
    df_rank = pd.merge(df_rank, df_append, on='日期', how='outer')

df_rank = pd.merge(df_rank, symbol.symbol_data[['日期', field]], on='日期', how='outer')

### 历史水位分析

基差率-库存消费比-利润率

历史时间比例分位

In [ ]:
# df1 = symbol.history_time_ratio('库存', df_rank=merged_data)
# df2 = symbol.history_time_ratio('库存', df_rank=merged_data, mode='value')


In [ ]:
# 将累计销量数据转化为当月销量数据

# 假设您提供的数据保存在一个名为df的dataframe中，字段包含日期和累计销量
# 例如，df的前五行如下：
#          日期   累计销量
# 0 2020-01-31  1000
# 1 2020-02-29  1500
# 2 2020-03-31  1800
# 3 2020-04-30  2200
# 4 2020-05-31  2500

# 定义一个函数，计算当月销量值
def calc_monthly_sales(df):
    # 创建一个空的列表，用于存储当月销量值
    monthly_sales = []
    # 遍历dataframe的每一行，获取日期和累计销量
    for i, row in df.iterrows():
        # 获取日期
        date = row['日期']
        # 获取累计销量
        cum_sales = row['累计销量']
        # 如果是第一行，那么需要判断是否是1月份
        if i == 0:
            # 如果是1月份，那么当月销量值就等于累计销量
            if date.month == 1:
                monthly_sales.append(cum_sales)
            # 如果不是1月份，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
        # 如果不是第一行，那么需要判断当前月份与上一行的月份是否相邻
        else:
            # 获取上一行的日期
            prev_date = df.loc[i-1, '日期']
            # 如果当前月份与上一行的月份相邻，那么当月销量值就等于累计销量减去上一行的累计销量
            if date.month == prev_date.month + 1 or (date.month == 1 and prev_date.month == 12):
                monthly_sales.append(cum_sales - df.loc[i-1, '累计销量'])
            # 如果当前月份与上一行的月份不相邻，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
    # 返回列表
    return monthly_sales

# 调用函数，得到一个列表，存储当月销量值
monthly_sales = calc_monthly_sales(df)

# 在原始的dataframe中，创建一个新的列，存储当月销量值
df['当月销量'] = monthly_sales

# 打印dataframe的前五行，查看结果
print(df.head())

#          日期   累计销量  当月销量
# 0 2020-01-31  1000   1000.0
# 1 2020-02-29  1500    500.0
# 2 2020-03-31  1800    300.0
# 3 2020-04-30  2200    400.0
# 4 2020-05-31  2500    300.0


## 季节性分析

### 基差率季节分析

In [ ]:
df_rb0['年度'] = df_rb0['日期'].dt.year
df_rb0['年内日期'] = df_rb0['日期'].dt.strftime('1900-%m-%d')
fig_basis_rate_season = px.line(df_rb0,
                                x='年内日期',
                                y='基差率',
                                color='年度',
                                #color_discrete_sequence=px.colors.qualitative.G10)
                                color_discrete_sequence=['lightgray', 'lightblue', 'orange', 'red'])
fig_basis_rate_season.update_layout(
    title={
        'text':'基差率季节分析',
        'xanchor':'center'},
    margin=dict(l=10, r=10, t=40, b=10)
)

fig_basis_rate_season.show()

### 基差率月度涨跌统计

### 基差率频率分布

### 库存季节性分析

## 跨期分析

### 期限结构

In [ ]:
# 加载合约基础数据
futures_comm_info = pd.read_excel('data/common_info.xlsx')
spec_contact_list = futures_comm_info[futures_comm_info.合约名称.str.startswith('螺纹钢')]
fig_term = make_subplots(specs=[[{"secondary_y": True}]])
fig_term.add_trace(go.Scatter(x=spec_contact_list['合约代码'], y=spec_contact_list['现价']))
# 获取最新现货价格
spot_price = df_rb0[df_rb0['现货']!=0]['现货'].iloc[-1]
fig_term.add_hline(y=spot_price)
fig_term.update_layout(
    title={
        'text':'期限结构'
    },
    #autosize=False,
    width=800,
    #height=800,
    margin=dict(l=10, r=10, t=40, b=10)
)
fig_term.show()

### 套利分析

#### 价差分析-多期排列

#### 价差分析-跨期价差矩阵

#### 基差-月差分析

#### 价差季节性分析

## 库存

### 库存周期

#### 期转现

#### 交割统计

## 利润

### 现货利润

### 期货盘面利润

### 利润期限结构

## 综合分析

### 基差-库存-利润分析

### 基差-月差分析

### 期限结构-库存/仓单分析